In [1]:
import tensorflow as tf
import scipy.io 
import matplotlib.pyplot as plt
import cv2
import keras
from glob import glob
import numpy as np
from tqdm import tqdm
import os
from PIL import Image
import pandas as pd
import cv2

from sklearn.model_selection import KFold
# from keras.preprocessing.image import ImageDataGenerator

# import keras_metrics

from keras.applications import mobilenet, resnet50 #, vgg16, inception_v3, resnet50, 
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, History

from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

import logging
# logging.getLogger().setLevel(logging.DEBUG)
import pickle


import seaborn
seaborn.set_style("darkgrid")

from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import precision_recall_curve

Using TensorFlow backend.


In [2]:
keras.__version__

'2.2.4'

## Params

In [3]:
# all_data_dir = 'E:\\Work/PathoBarIlan/Shlomi2018/'
all_data_dir = '/media/leetwito/DATA/Datasets/PathoBarIlan/Shlomi2018'

is_relative_path_csv = False
seed = 4221
k_idx = 3 # take one of the K-Folds
pretrained_model_path = "/media/leetwito/Windows/Users/leetw/PycharmProjects/PathoBarIlan/my_models/k=3, lr1e-2/model_spec_weights_epoch36-val_loss0.006-train_loss0.004-seed4221-i=3.hdf5"


pos_name_init = 'Cancer'
neg_name_init = 'Normal'

use_rgb = False # True=rgb, False=spectral
if use_rgb:
    file_ext = '.png'
else:
    file_ext = '.npy'
    
window_size = (200, 200)
shift = (100, 100)

In [4]:
w,h = window_size
if use_rgb:
    input_shape = (w,h,3)
else:
    input_shape = (w,h,40)
batch_size = 16

## utils

In [5]:
def read_slide(path):
    mat = scipy.io.loadmat(path)
    spectral = mat["Spec"]
    rgb = mat["Section"]
    shape = rgb.shape
    
    return spectral, rgb

In [6]:
def visualize_batch_of_crops(crops, n_iter_y, n_iter_x):
    fig, axes = plt.subplots(n_iter_y, n_iter_x, figsize=(5, 5), gridspec_kw = {'wspace':0, 'hspace':0})

    for i in range(n_iter_x):
        for j in range(n_iter_y):
            axes[j, i].imshow(crops[i*n_iter_y + j])
            axes[j, i].axis('off')
            axes[j, i].set_aspect('equal')
    plt.show()

In [7]:
def create_csv_for_folder(data_dir, ext):
    if ext[0] == '.':
        ext = ext[1:]
    data_df = pd.DataFrame(columns=['filename', 'label'])
    files = glob(os.path.join(data_dir,'*', '*.{}'.format(ext)))
    files = [file for file in files if "Mixed" not in file]
#     print(data_dir+'/*/*.{}'.format(ext))
    
    init_len = len(data_dir)
    delete_folder = all_data_dir
    if not is_relative_path_csv:
        delete_folder = '/'
    if not delete_folder[-1] == '/':
        delete_folder += '/'
    files = [file.replace(delete_folder, '/') for file in files]
#     print(files)
    labels = [1 if pos_name_init in file else 0 for file in files]
#     print(labels)
    data_df['filename'] = files
    data_df['label'] = labels
#     data_df.to_csv(os.path.join(data_dir, os.path.basename(data_dir)+'.csv'), index=False)
#     print('Created CSV successfully for folder {}'.format(data_dir))
    
    return data_df    

In [8]:
slides = glob(os.path.join(all_data_dir, "*/"))
slides

['/media/leetwito/DATA/Datasets/PathoBarIlan/Shlomi2018/Case10/',
 '/media/leetwito/DATA/Datasets/PathoBarIlan/Shlomi2018/Case11/',
 '/media/leetwito/DATA/Datasets/PathoBarIlan/Shlomi2018/Case12/',
 '/media/leetwito/DATA/Datasets/PathoBarIlan/Shlomi2018/Case14/',
 '/media/leetwito/DATA/Datasets/PathoBarIlan/Shlomi2018/Case16/',
 '/media/leetwito/DATA/Datasets/PathoBarIlan/Shlomi2018/Case16b/',
 '/media/leetwito/DATA/Datasets/PathoBarIlan/Shlomi2018/Case17/',
 '/media/leetwito/DATA/Datasets/PathoBarIlan/Shlomi2018/Case18/',
 '/media/leetwito/DATA/Datasets/PathoBarIlan/Shlomi2018/Case19484/',
 '/media/leetwito/DATA/Datasets/PathoBarIlan/Shlomi2018/Case8/']

In [9]:
skf = KFold(n_splits=5, shuffle=True, random_state=seed)

train_slides_all = []
test_slides_all = []
val_slides_all = []

for train_index, test_index in skf.split(np.arange(len(slides)).T, np.arange(len(slides)).T):
    print("TRAIN:", train_index, "TEST:", test_index)
    train_slides_all.append(train_index)
    val_slides_all.append([test_index[0]])
    test_slides_all.append([test_index[1]])

TRAIN: [2 3 4 5 6 7 8 9] TEST: [0 1]
TRAIN: [0 1 2 3 5 7 8 9] TEST: [4 6]
TRAIN: [0 1 3 4 5 6 7 8] TEST: [2 9]
TRAIN: [0 1 2 4 6 7 8 9] TEST: [3 5]
TRAIN: [0 1 2 3 4 5 6 9] TEST: [7 8]


In [10]:
train_index = train_slides_all[k_idx]
val_index = val_slides_all[k_idx]
test_index = test_slides_all[k_idx]

train_index, val_index, test_index

(array([0, 1, 2, 4, 6, 7, 8, 9]), [3], [5])

In [11]:
def get_dfs_for_indices(slides, index_list):
    dfs = []
    for slide in np.array(slides)[index_list]:
        data_dir = slide
        dfs.append(create_csv_for_folder(data_dir, file_ext))
    df = pd.concat(dfs, ignore_index=True)
    df = df.sample(frac=1, random_state=seed)  # frac=1 is same as shuffling df.
    return df

In [12]:
df_train = get_dfs_for_indices(slides, train_index)
df_test = get_dfs_for_indices(slides, test_index)
df_val = get_dfs_for_indices(slides, val_index)

In [13]:
pd.options.display.max_colwidth = 150

In [14]:
assert len(set(df_train.label.values)) == 2 and len(set(df_val.label.values)) == 2 and len(set(df_test.label.values)) == 2  

In [15]:
def batch_norm(x):
#     print("x.shape:", x.shape)
    maxi = x.max(axis=1).max(axis=1)
#     print("maxi.shape:", maxi.shape)
    maxi = np.repeat(maxi[:, np.newaxis, : ], window_size[0], axis=1)
    maxi = np.repeat(maxi[:, np.newaxis, : ], window_size[1], axis=1)
    return x/maxi


def generator_from_df(df, batch_size, shuffle=True): 
    
    n_batches = df.shape[0]//batch_size
    while True:
        if shuffle:
            df_tmp = df.copy().sample(frac=1)  # frac=1 is same as shuffling df.
        else:
            df_tmp = df
        
        for i in range(n_batches):
            sub = df_tmp.iloc[batch_size*i:batch_size*(i+1)]
            if use_rgb:
                X = [img_to_array(load_img(f, target_size=input_shape)) for f in sub.filename]
            else:
                X = [np.load(f) for f in sub.filename]
                
            X = batch_norm(np.stack(X))
            logging.debug("from file {}\nto file {}".format(sub.iloc[0].filename, sub.iloc[-1].filename))

            Y = sub.label.values
            Y = to_categorical(Y, num_classes=2)
            # Simple model, one input, one output.
            
            yield X, Y

In [23]:
def plot_roc_curve(y_true, y_scores, figsize=(15, 8)):
    fpr, tpr, threshold = roc_curve(y_true, y_scores)
    plt.figure(figsize=figsize)
    plt.plot(fpr, tpr)
    plt.plot([0, 1], [0, 1], "k--")
    plt.axes()

    return fpr, tpr, threshold

In [16]:
train_generator = generator_from_df(df_train, batch_size)
val_generator = generator_from_df(df_val, batch_size)
test_generator = generator_from_df(df_test, batch_size, shuffle=False)

In [17]:
loaded_model = keras.models.load_model(pretrained_model_path)

In [18]:
generator = train_generator
df = df_train

data_len = len(df)//batch_size
y_proba = []
y_gt = []
loss = []
for i in tqdm(range(data_len)):
    x, y = next(generator)
    y_gt.append(y)
    y_proba.append(loaded_model.predict(x))
    loss.append(loaded_model.evaluate(x, y))
with open(pretrained_model_path.replace('.hdf5', '-[y_gt,y_proba,loss].pkl'), 'wb') as f:
    pickle.dump([y_gt, y_proba, loss], f)

  0%|          | 0/187 [00:00<?, ?it/s]

16/16 [==============================] - 1s 36ms/step


  1%|          | 1/187 [00:04<15:22,  4.96s/it]

16/16 [==============================] - 0s 15ms/step


  1%|          | 2/187 [00:08<13:56,  4.52s/it]

...

 99%|█████████▉| 185/187 [11:21<00:07,  3.62s/it]

16/16 [==============================] - 0s 15ms/step


 99%|█████████▉| 186/187 [11:25<00:03,  3.64s/it]

16/16 [==============================] - 0s 14ms/step


100%|██████████| 187/187 [11:28<00:00,  3.61s/it]


In [19]:
y_proba = np.concatenate(y_proba)
y_gt = np.concatenate(y_gt)

In [20]:
results = y_proba.argmax(axis=1)==y_gt.argmax(axis=1)

In [21]:
print("correct: {}/{}".format(results.sum(), len(results)))

correct: 2992/2992


In [25]:
y_pred = np.argmax(y_proba, axis=1)
plot_roc_curve(y_gt, y_pred)
plt.show()

ValueError: multilabel-indicator format is not supported

In [27]:
y_gt.argmax(axis=1)

array([1, 0, 0, ..., 0, 0, 1])

In [31]:
y_proba.shape

(2992, 2)

In [36]:
scores = [y_proba[idx, pred] for idx, pred in enumerate(y_gt.argmax(axis=1))]

In [37]:
low_score_pred = sorted(scores)

In [43]:
low_score_pred[:15]

[0.81256026,
 0.8556575,
 0.85801196,
 0.86075246,
 0.8644029,
 0.86545503,
 0.8771054,
 0.8848943,
 0.88885045,
 0.893748,
 0.8950029,
 0.90370256,
 0.9097792,
 0.92767656,
 0.9307996]

In [ ]:
y_pred = np.argmax(y_pred, axis=1)
y_pred.shape

In [ ]:
y_pred.sum()

In [ ]:
y_test.sum()

In [ ]:
(y_pred==y_test).sum()/220

### plot roc curve

In [ ]:
def plot_roc_curve(y_true, y_scores, figsize=(15, 8)):
    fpr, tpr, threshold = roc_curve(y_true, y_scores)
    plt.figure(figsize=figsize)
    plt.plot(fpr, tpr)
    plt.plot([0, 1], [0, 1], "k--")
    plt.axes()

    return fpr, tpr, threshold

In [ ]:
def plot_precision_recall_curve(y_true, y_scores, figsize=(15, 8)):
    precision, recall, thresholds = precision_recall_curve(y_true, y_scores)
    plt.figure(figsize=figsize)
    plt.plot(precision, recall)
#     plt.plot([0, 1], [0, 1], "k--")
    plt.axes()
    
    return precision, recall, thresholds

In [ ]:
# mobilenet_model.load_weights("my_models/model_spec_weights_epoch36-val_loss0.006-train_loss0.004.hdf5")
mobilenet_model.load_weights("my_models/model_spec_weights_epoch14-val_loss0.019-seed4221-k_idx=1.hdf5")

In [ ]:
def get_y_test_and_pred():
    y_tests = []
    y_preds = []
    for i in range(len(df_test)//batch_size):
        print(i, "out of", len(df_test)//batch_size)
        x, y = next(test_generator)
        print(mobilenet_model.evaluate(x, y))
        y_tests.append(y.argmax(axis=1))
        y_preds.append(mobilenet_model.predict(x)[:,1])
    
    y_test = np.stack(y_tests)
    y_pred = np.stack(y_preds)

    y_test = y_test.reshape((-1, 1))
    y_pred = y_pred.reshape((-1, 1))
    
    return y_test, y_pred

In [ ]:
y_test, y_pred = get_y_test_and_pred()

In [ ]:
print(roc_auc_score(y_test, y_pred))
fpr, tpr, threshold = plot_roc_curve(y_test, y_pred, figsize=(15, 8))

In [ ]:
# precision, recall, threshold = plot_precision_recall_curve(y_test, y_pred, figsize=(15, 8)) # kills the kernel for me.